# Wildcat Creek Flood Inundation Mapping Segment-by-Segment

This notebook uses the Wildcat Creek (near Manhattan, KS) to demonstarte how to create segment-by-segment inundation map for developing FLDPLN-based synthetic rating curve (SRC) for converting the NWM discharge into stage.

## Import Modules

Import necessary modules.

In [ ]:
import sys
import time
from dask.distributed import Client, LocalCluster
from dask import visualize

### Import FLDPLN Modules

In [ ]:
# Tool/script folder
fldplnToolFolder = r'Z:\FLDPLN\tools_os' # tool development folder, has the latest version

# Add the tool/script folder to sys.path to access fldpln modules
sys.path.append(fldplnToolFolder) 
# fldpln module
from fldpln import *
from fldpln_library import *
# from fldpln_gauge import *

## Setup Input Tiled Library and Output Folders

Here we setup the folder under which tiled libraries (organized as folders) are located. We also setup the output folder (i.e., outputFolder) under which a map folder and a 'scratch' folder are created. The map folder, which is specified later, comtains all inundation depth maps. The scratch folder stores temporary files.

In [ ]:
# tiled library folder
# libFolder =  r'E:\fldpln\sites\wildcat_10m_3dep\tiled_snz_library' # Wildcat
libFolder =  r'E:\fldpln\sites\verdigris_10m\tiled_snz_library' # Cerdigris
# library to map inundation
libName = 'lib'

# Set output folder
outputFolder = r'E:\fldpln\sites\verdigris_10m\maps'

## Specify Segment and Stage/DOF

Based on the segment ID provided, we retrieve the FSPs on the segment and assign the FSPs with a constant stage as specified.

In [12]:
# specify segment ID and the stage/DOF
segId=337; dof=25

# Read in library FSP info CSV file
fspFile = os.path.join(libFolder, libName, fspInfoFileName)
fspDf = pd.read_csv(fspFile) 
# print(fspDf)

# select the FSPs on the segment
segFsps = fspDf[fspDf['SegId']==segId]
# print(segFsps)

# prepare the FSPs for mapping
segFsps.loc[:,'LibName'] = libName
segFsps.loc[:,'Dof'] = dof
fspDof = segFsps[['LibName','FspId','Dof']]
print(fspDof)

    LibName  FspId  Dof
0       lib      1   25
1       lib      2   25
2       lib      3   25
3       lib      4   25
4       lib      5   25
..      ...    ...  ...
317     lib    318   25
318     lib    319   25
319     lib    320   25
320     lib    321   25
321     lib    322   25

[322 rows x 3 columns]


C:\Users\lixi\AppData\Local\Temp\ipykernel_10380\3950496016.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  segFsps.loc[:,'LibName'] = libName
C:\Users\lixi\AppData\Local\Temp\ipykernel_10380\3950496016.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  segFsps.loc[:,'Dof'] = dof


## Map Flood Inundation Depth


### Set Mapping Parameters

Setup the map folder (i.e., outMapFolderName) which is under the output folder and comtains all inundation depth maps. Additional settings include whether to mosaic tiles as single COG file and whether use a Dask local cluster to speed up the mapping.

In [13]:
# set up map folder
outMapFolderName = 'segment_'+str(segId)+'_dof_'+str(dof)

# Create folders for storing temp and output map files
outMapFolder,scratchFolder = CreateFolders(outputFolder,'scratch',outMapFolderName)

# whether mosaci tiles as a single COG
mosaicTiles = True #True #False

# Using LocalCluster by default
useLocalCluster = False # This doesn't work on my office desktop though it works fine on KBS server
numOfWorkers = round(0.8*os.cpu_count())
numOfWorkers = 6
print(f'Number of workers: {numOfWorkers}')

Number of workers: 6


### Map Inundation Depth

The process of generating inundation depth map happens here.

In [14]:
# show mapping info
print(f'Tiled FLDPLN library folder: {libFolder}')
print(f'Map folder: {outMapFolder}')
# Find libs needs mapping
libs2Map = fspDof['LibName'].drop_duplicates().tolist()
print(f'Libraries to map: {libs2Map}')

# check running time
startTimeAllLibs = time.time()

# create a local cluster to speed up the mapping. Must be run inside "if __name__ == '__main__'"!!!
if useLocalCluster:
    # cluster = LocalCluster(n_workers=4,processes=False)
    try:
        print('Start a LocalCluster ...')
        # NOTE: set worker space (i.e., local_dir) to a folder that the LocalCluster can access. When run the script through a scheduled task, 
        # the system uses C:\Windows\system32 by default, which a typical user doesn't have the access!
        # cluster = LocalCluster(n_workers=numOfWorkers,memory_limit='32GB',local_dir="D:/projects_new/fldpln/tools") # for KARS production server (192G RAM & 8 cores)
        # cluster = LocalCluster(n_workers=numOfWorkers,processes=False) # for KARS production server (192G RAM & 8 cores)
        cluster = LocalCluster(n_workers=numOfWorkers,memory_limit='8GB',local_dir="E:\temp") # for office desktop (64G RAM & 8 cores)
        # print('Watch workers at: ',cluster.dashboard_link)
        print(f'Number of workers: {numOfWorkers}')
        client = Client(cluster)
        # print scheduler info
        # print(client.scheduler_info())
    except:
        print('Cannot create a LocalCLuster!')
        useLocalCluster = False

# dict to store lib processing time
libTime={}

# map each library
for libName in libs2Map:
    # check running time
    startTime = time.time()
    
    # select the FSPs within the lib
    fspIdDof = fspDof[fspDof['LibName']==libName][['FspId','Dof']]

    # mapping flood depth
    if useLocalCluster:
        print(f'Map [{libName}] using LocalCLuster ...')
        # generate a DAG
        dag,dagRoot=MapFloodDepthWithTilesAsDag(libFolder,libName,'snappy',outMapFolder,fspIdDof,aoiExtent=None)
        if dag is None:
            tileTifs = None
        else:
            # visualize DAG
            # visualize(dag)
            # Compute DAG
            tileTifs = client.get(dag, dagRoot)
            if not tileTifs: # list is empty
                tileTifs =  None
    else:
        print(f'Map {libName} ...')
        tileTifs = MapFloodDepthWithTiles(libFolder,libName,'snappy',outMapFolder,fspIdDof,aoiExtent=None)
    print(f'Actual mapped tiles: {tileTifs}')

    # Mosaic all the tiles from a library into one tif file
    if mosaicTiles and not(tileTifs is None):
        print('Mosaic tile maps ...')
        mosaicTifName = libName+'_'+outMapFolderName+'.tif'
        # Simplest implementation, may crash with very large raster
        MosaicGtifs(outMapFolder,tileTifs,mosaicTifName,keepTifs=False)
    
    # check time
    endTime = time.time()
    usedTime = round((endTime-startTime)/60,3)
    libTime[libName] = usedTime
    # print(f'{libName} processing time (minutes):', usedTime)

# Show processing time
# Individual lib processing time
print('Individual library mapping time:', libTime)
# total time
endTimeAllLibs = time.time()
print('Total processing time (minutes):', round((endTimeAllLibs-startTimeAllLibs)/60,3))

#
# Shutdown local clusters
#
if useLocalCluster:
    print('Shutdown LocalCluster ...')
    cluster.close()
    client.shutdown()
    client.close()
    useLocalCluster = False

Tiled FLDPLN library folder: E:\fldpln\sites\verdigris_10m\tiled_snz_library
Map folder: E:\fldpln\sites\verdigris_10m\maps\segment_337_dof_25
Libraries to map: ['lib']
Map lib ...
Tiles need to be mapped: [3]
Actual mapped tiles: ['E:\\fldpln\\sites\\verdigris_10m\\maps\\segment_337_dof_25\\lib_tile_3.tif']
Mosaic tile maps ...
Individual library mapping time: {'lib': 0.008}
Total processing time (minutes): 0.008
